## Environnements RL

In [1]:
# HIDDEN

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Qu'est-ce qu'un environnement ?

- Un environnement peut être :
  - un jeu, comme un jeu vidéo.
  - une simulation d'un scénario du monde réel, comme un robot, le comportement d'un utilisateur ou le marché boursier
  - toute autre configuration avec un _agent_ qui prend des _actions_, voit des _observations_ et reçoit des _récompenses_
  
Une note sur la terminologie : nous utiliserons indifféremment _agent_ et _joueur_ 

#### Exemple de course : lac gelé

Comme exemple de fonctionnement d'un environnement, nous allons utiliser l'environnement [Frozen Lake](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/) de [OpenAI Gym](https://www.gymlibrary.ml/), qui fournit l'interface standard pour les problèmes RL. Nous pouvons visualiser l'environnement comme suit :

In [2]:
import gym
env = gym.make("FrozenLake-v1", is_slippery=False)

In [3]:
# HIDDEN
from utils_01 import fix_frozen_lake_render
fix_frozen_lake_render(env)

In [4]:
env.reset()
env.render()


P...
.O.O
...O
O..G


Le but est que le joueur (`P`) atteigne le but (`G`) en marchant sur les segments du lac gelé (`.`) sans tomber dans les trous (`O`).

**Note** : le rendu de l'environnement a été modifié par rapport au rendu d'OpenAI Gym afin qu'il puisse être affiché clairement dans cette plateforme d'apprentissage interactive.

Notes :

Le code n'est pas montré pour le remplacement du rendu.

#### Mouvement

Le joueur peut se déplacer sur le lac gelé. Par exemple :

In [5]:
env.step(1) # 1 -> Down
env.render()

  (Down)
....
PO.O
...O
O..G


Ne t'inquiète pas de `step(1)` pour l'instant ; nous y reviendrons 

Ce que tu peux voir, c'est que le joueur (`P`) s'est déplacé vers le bas.

#### Objectif

Avance d'un grand nombre d'étapes et tu as terminé le puzzle :

In [6]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.render()

  (Right)
....
.O.O
...O
O..P


Tu as atteint l'objectif en atteignant la partie inférieure droite.

#### Qu'est-ce qui fait un environnement ?

Un environnement comprend plusieurs composants clés, que nous allons passer en revue dans les diapositives suivantes.

#### États

- Nous utiliserons le terme _état_ de manière informelle pour désigner tout ce qui concerne l'environnement 

In [7]:
env.reset()
env.render()


P...
.O.O
...O
O..G


- Par exemple, voici l'état de départ de l'environnement.
- Le joueur se trouve en haut à gauche, il y a de la glace gelée à proximité, etc.
- Nous utiliserons le concept d'état pour parler de notre environnement, mais il n'apparaîtra pas dans l'"API".

#### Actions

- Ici, le joueur peut choisir entre 4 actions possibles : gauche, bas, droite, haut
- L'espace de toutes les actions possibles s'appelle l' **espace des actions**.
- Dans SL, nous distinguons la régression ($y$ continu) et la classification ($y$ catégorique)
- De même, dans RL, l'espace d'action peut être continu ou discret
- Dans ce cas, il est discret (4 possibilités)
- Le code est d'accord :

In [8]:
env.action_space

Discrete(4)

#### Observations

- Les observations sont les _parties de l'état que l'agent peut voir_.
- Parfois, l'agent peut tout voir ; nous appelons cela _tout à fait observable_.
- Souvent, nous avons des environnements _partiellement observables_ 
- Dans l'exemple du lac gelé, l'agent ne peut voir que son propre emplacement parmi les 16 carrés.
- On ne "dit" pas à l'agent où se trouvent les trous par le biais d'observations directes, il devra donc _apprendre_ cela par essais et erreurs.

#### Observations

- L'espace de toutes les observations possibles s'appelle l' **espace d'observation**.
- Tu peux considérer l'espace d'action comme analogue à la cible dans l'apprentissage supervisé.
- Tu peux considérer l'espace d'observation comme analogue aux caractéristiques de l'apprentissage supervisé.


Ici, nous avons un espace d'observation discret composé des 16 positions possibles du joueur :

In [9]:
env.observation_space

Discrete(16)

#### Récompenses

- Dans l'apprentissage supervisé, l'objectif est généralement de faire de bonnes prédictions.
- Tu peux toujours essayer différentes fonctions de perte en fonction de ton objectif spécifique, mais le concept général est le même.
- En RL, l'objectif peut être n'importe quoi.
- Mais, comme dans SL, tu devras _optimiser_ quelque chose.
- En RL, nous cherchons à maximiser la **récompense**.

#### Récompenses 

Dans l'exemple de Frozen Lake, l'agent reçoit une récompense lorsqu'il atteint l'objectif.

In [10]:
env.reset()
obs, reward, done, _ = env.step(0)
env.render()
print("reward =", reward)

  (Left)
P...
.O.O
...O
O..G
reward = 0.0


In [11]:
obs, reward, done, _ = env.step(1)
print("reward =", reward)

reward = 0.0


Toujours pas de récompense, continuons...

#### Récompenses

In [12]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
obs, reward, done, _ = env.step(2)
env.render()
print("reward =", reward)

  (Right)
....
.O.O
...O
O..P
reward = 1.0


Nous avons reçu une récompense de 1.0 pour avoir atteint l'objectif.

#### Boucle environnement-agent

Remarque comment l'agent (dans ce cas, nous) et l'environnement communiquent dans une boucle aller-retour :

[](img/RL-loop.png)

C'est le diagramme classique que tu verras partout.

L'objectif sera d'apprendre automatiquement le comportement de l'agent (reste à l'écoute !).

#### Représenter des actions

- Pour utiliser le logiciel RL, nous aurons besoin d'une représentation numérique de notre espace d'action et de notre espace d'observation.
- Dans ce cas, nous avons 4 actions discrètes possibles, nous pouvons donc les coder sous la forme {0,1,2,3} pour (gauche, bas, droite, haut).
- C'est pourquoi, plus tôt, nous avons fait

In [13]:
env.step(1)

pour marcher vers le bas.

#### Représenter des observations

- De même, nous aurons besoin d'une représentation numérique de nos observations.
- Ici, il y a 16 positions possibles du joueur. Elles sont codées de 0 à 15 comme suit :

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Ces détails sur l'environnement Frozen Lake sont également disponibles dans la [documentation] (https://www.gymlibrary.ml/environments/toy_text/frozen_lake/).

#### Représenter des observations

Au départ, nous observons "0" car nous commençons en haut à gauche :

In [14]:
env.reset()
env.render()


P...
.O.O
...O
O..G


Après s'être déplacé vers le bas (action 1), nous nous déplaçons vers la position 4.

In [15]:
obs, reward, done, _ = env.step(1)
obs

4

L'observation est renvoyée par la méthode `step()`.

#### Environnements non déterministes

- Jusqu'à présent, entreprendre une action particulière à partir d'un état particulier a toujours abouti au même nouvel état.
- En d'autres termes, notre environnement Frozen Lake était _déterministe_.
- Certains environnements sont _non-déterministes_, ce qui signifie que le résultat d'une action peut être aléatoire.
- Nous pouvons initialiser un Frozen Lake non déterministe comme ceci :

In [16]:
env_slippery = gym.make("FrozenLake-v1", is_slippery=True)

In [17]:
# HIDDEN
env_slippery.seed(4)
fix_frozen_lake_render(env_slippery)

In [18]:
env_slippery.reset()
env_slippery.render()


P...
.O.O
...O
O..G


#### Environnements non déterministes

In [19]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
PO.O
...O
O..G


Le déménagement n'a pas fonctionné comme prévu.

In [20]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Le déplacement vers le bas a fonctionné cette fois.

Dans cet environnement "glissant" de Frozen Lake, le mouvement ne fonctionne comme prévu qu'un tiers du temps.

#### Episodes

- Jouer au lac gelé a une fin - soit tu tombes dans un trou, soit tu atteins l'objectif.
- Cependant, un seul jeu n'est pas suffisant pour qu'un algorithme RL puisse en tirer des leçons.
- Il aura besoin de plusieurs sessions de jeu, appelées **épisodes**.
- Après un épisode, l'environnement est réinitialisé.

#### Episodes

La méthode `step()` renvoie un drapeau nous indiquant si l'épisode est terminé :

In [21]:
obs, reward, done, _ = env_slippery.step(1)
done

True

In [22]:
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Ici, l'épisode est terminé parce que nous sommes tombés dans un trou.

#### Episodes

- Dans certains environnements (comme Frozen Lake), les récompenses ne sont reçues qu'à la fin d'un épisode.
- Dans d'autres environnements, les récompenses peuvent être reçues à n'importe quel **pas de temps** (c'est-à-dire après une action).

#### Mettre tout en place

- Nous avons maintenant parlé des principaux composants d'un environnement RL :
  - États
  - Actions
  - Observations
  - Récompenses
  - Épisodes
  

#### Jeux de données SL vs. environnements RL

- Dans l'apprentissage supervisé, on te donne généralement un ensemble de données.
- En RL, l'environnement agit comme un _générateur de données_.
  - Plus tu joues dans l'environnement, plus tu génères de "données" et plus tu peux apprendre.
- On peut aussi faire du RL sur un ensemble de données pré-collectées (appelé _RL hors ligne_), mais cela ne nous concerne pas.

#### Appliquons ce que nous avons appris !

## Environnement de la voiture auto-conductrice
<!-- multiple choice -->

Tu utilises RL pour entraîner une voiture à conduite autonome. L'IA de la voiture utilise diverses caméras et capteurs comme entrées et doit décider de l'angle du volant ainsi que de l'angle des pédales d'accélérateur/frein au sol.

#### L'espace d'observation est-il continu ou discret ?

- [x] Continu
- [ ] Discret | Dans ce cas, les observations sont les entrées du capteur, par exemple les estimations de la profondeur.

#### L'espace d'action est-il continu ou discret ?

- [x] Continu
- [ ] Discret | Les actions sont des angles ; elles ne proviennent pas d'un ensemble discret d'options.

#### Quelle serait la structure de récompense la plus raisonnable pour cet environnement ?

- [La récompense est égale à la durée pendant laquelle la voiture a pu rouler sans s'écraser | Quelle serait la récompense si la voiture ne bouge jamais ?
- [ ] La récompense est égale à la distance que la voiture a pu parcourir sans s'écraser | Oui, ça sonne bien !
- [ ] +1 récompense chaque fois que la voiture s'écrase | N'oublie pas que nous voulons maximiser la récompense, pas la minimiser.

## Épisodes en fonction des pas de temps
<!-- multiple choice -->

Remplis les espaces vides de la phrase suivante 

*Dans un environnement d'apprentissage par renforcement, on effectue des actions de manière répétée jusqu'à ce que l'\_\_\_\_\_\_\_s'arrête. Cela peut impliquer une seule \_\_\_\_\_\_\_, ou de très nombreuses.*

- [ ] pas de temps / récompense | Vérifie bien le premier blanc !
- [ ] récompense / étape de temps | Essaie encore !
- [ ] étape temporelle / épisode | Essaie encore !
- [ ] épisode / étape de temps | Tu l'as eu !


## L'environnement du taxi de Gym
<!-- coding exercise -->

Dans cet exercice, nous allons examiner l'un des environnements textuels fournis avec
OpenAI gym, appelé l'environnement taxi.
Documentation [ici](https://www.gymlibrary.ml/environments/toy_text/taxi/).

In [33]:
# EXERCISE
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()

# Add calls to `step` here!

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Dans cet exercice, le taxi est représenté par la surbrillance jaune,
qui se trouve actuellement en bas à gauche de l'arène. Le `:` peut être traversé mais pas le `|`.
Le but est de prendre des passagers et de les déposer.

Il y a 6 actions possibles :
descendre (0), monter (1), droite (2), gauche (3), ramasser (4), déposer (5).

In [36]:
taxi.action_space

Discrete(6)

Le code suivant imprime l'observation dans un format plus lisible pour l'homme :

In [37]:
print("Taxi row: %d\nTaxi col: %d\nPassenger loc: %d\nDestination loc: %d" % tuple(taxi.decode(obs)))

Taxi row: 4
Taxi col: 0
Passenger loc: 0
Destination loc: 2


Les emplacements possibles sont `R` (0), `G` (1), `Y` (2), `B` (3) et en taxi (4). Ainsi, le passager se trouve actuellement à `R` et se dirige vers `Y`.

Pour répondre à la question suivante, tu devras modifier le code, l'exécuter et imprimer tout résultat pertinent.

In [42]:
# SOLUTION
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.render()
taxi.step(4)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.render()
obs, reward, done, _ = taxi.step(5)
print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
20


In [27]:
# TODO: in this case there is no testing of the code; the code is only used to explore
# this may not work well with the framework - add an automated test to the code as well? 
# e.g. can check that the state of the env is correct?

#### Quelle récompense l'agent reçoit-il pour avoir réussi à déposer le passager ?

- [ ] 0 | Essaie d'effectuer les actions nécessaires avec `taxi.step()` et d'imprimer les récompenses.
- [ ] 5 | Essaie d'effectuer les actions nécessaires avec `taxi.step()` et d'imprimer les récompenses.
- [ ] 10 | Essaie d'effectuer les actions nécessaires avec `taxi.step()` et d'imprimer les récompenses.
- [ ] 20 | Tu as réussi !

## Observations vs. rendus
<!-- coding exercise -->

L'environnement du lac gelé nous permet de créer un rendu visuel de l'environnement, que nous avons vu précédemment. Ce rendu est destiné à des fins humaines/de débogage, et n'est _pas_ vu par l'agent/algorithme. Ta tâche ici est de jouer un environnement Frozen Lake donné **sans regarder le rendu visuel** (pas de triche !). Remplis la liste `actions` pour qu'elle contienne un ensemble d'actions qui amènent correctement l'agent au but. Ce que tu vis est ce qu'un algorithme RL "voit" lorsqu'il apprend !

Note que l'environnement donné de Frozen Lake est 3x3 au lieu de 4x4. Ainsi, l'espace d'observation va de 0 à 8 au lieu de 0 à 15.

In [28]:
# EXERCISE

import gym
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", 
               desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), 
               is_slippery=False)
env.render = None

obs = env.reset()
actions = ____
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)


NameError: name '____' is not defined

In [6]:
# SOLUTION

import gym.envs.toy_text
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.render = None

obs = env.reset()
actions = []
# BEGIN SOLUTION
actions = [1,2,1,2]
# END SOLUTION
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)

Obs: 3 Reward: 0.0 Done: False
Obs: 4 Reward: 0.0 Done: False
Obs: 7 Reward: 0.0 Done: False
Obs: 8 Reward: 1.0 Done: True


In [9]:
# HIDDEN
import numpy as np
import gym.envs.toy_text
from utils_01 import fix_frozen_lake_render
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.reset()
fix_frozen_lake_render(env)
env.render()


PO.
...
O.G


#### Meilleur chemin vers l'objectif

En rappelant que les actions 0, 1, 2, 3 représentent respectivement la gauche, le bas, la droite et le haut, laquelle des affirmations suivantes décrit correctement le meilleur chemin vers le but ?

- [ ] Commence par te déplacer vers le bas, puis vers le bas à nouveau
- [ ] Commence par te déplacer vers le bas, puis vers la droite
- [ ] Commence par te déplacer vers la droite, puis à nouveau vers la droite
- [ ] Commence par te déplacer vers la droite, puis vers le bas